In [1]:
import gzip
import json
import pandas as pd
from my_functions_improved import *

c:\Users\cate9\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\cate9\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


‎𐤀 CLTK version '1.3.0'. When using the CLTK in research, please cite: https://aclanthology.org/2021.acl-demo.3/

Pipeline for language 'Ancient Greek' (ISO: 'grc'): `GreekNormalizeProcess`, `GreekSpacyProcess`, `GreekEmbeddingsProcess`, `StopsProcess`.

⸖ ``GreekSpacyProcess`` using OdyCy model by Center for Humanities Computing Aarhus from https://huggingface.co/chcaa . Please cite: https://aclanthology.org/2023.latechclfl-1.14
⸖ ``LatinEmbeddingsProcess`` using word2vec model by University of Oslo from http://vectors.nlpl.eu/ . Please cite: https://aclanthology.org/W17-0237/

⸎ To suppress these messages, instantiate ``NLP()`` with ``suppress_banner=True``.


In [2]:
with gzip.open('noun_dict.json.gz', 'rt', encoding='utf-8') as gzip_file:
    noun_dict = json.load(gzip_file)

In [3]:
with gzip.open('verb_dict.json.gz', 'rt', encoding='utf-8') as gzip_file:
    verb_dict = json.load(gzip_file)

In [61]:
noun_df = pd.DataFrame(noun_dict)

In [5]:
verb_df = pd.DataFrame(verb_dict)

In [6]:
noun_df

,lemma,form,gender,case,number,dialects
0,ἅβρα,ἅβρα,fem,nom/voc/acc,dual,
1,ἅβρα,ἅβρα,fem,nom/voc,sg,"[(attic, doric, aeolic]"
2,ἅβρα,ἅβραι,fem,nom/voc,pl,
3,ἅβρα,ἅβραι,fem,dat,sg,"[(attic, doric, aeolic]"
4,ἅβρα,ἅβραις,fem,dat,pl,
...,...,...,...,...,...,...
216247,ἵστωρ,ῐ̔́στορες,,nom,pl,
216248,ἵστωρ,ῐ̔́στωρ,,nom,sg,
216249,ἵστωρ,ῐ̔́στορε,,voc,dual,
216250,ἵστωρ,ῐ̔́στορες,,voc,pl,


In [7]:
verb_df

,lemma,form,tense,mode,act/mid/p,gender,case,person,number,dialects
0,ἅλλομαι,ἅλεται,aor,subj,mid,,,3rd,sg,
1,ἅλλομαι,ἅληται,aor,subj,mid,,,3rd,sg,
2,ἅλλομαι,ἅλῃ,aor,subj,mid,,,2nd,sg,
3,ἅλλομαι,ἅλλεσθαι,pres,inf,mid,,,,,
4,ἅλλομαι,ἅλλεσθε,imperf,ind,mid,,,2nd,pl,"[(doric, aeolic]"
...,...,...,...,...,...,...,...,...,...,...
859327,ζῳοτροφέω,ζῳοτροφεῖν,pres,inf,act,,,,,[doric]
859328,ζῳοτροφέω,ζῳοτροφούντων,pres,part,act,masc/neut,gen,,pl,[doric]
859329,ζῳοτροφέω,ζῳοτροφούντων,pres,imperat,act,,,3rd,pl,[doric]
859330,ζῳοτροφέω,ζῳοτροφοῦσιν,pres,part,act,masc/neut,dat,,pl,[doric]


In [15]:
null_df = verb_df[verb_df["tense"] == " "]
null_df

,lemma,form,tense,mode,act/mid/p,gender,case,person,number,dialects
12131,ἀφίστημι,ἀφίσταιντ’,,,,,,3rd,pl,[(ionic]
12132,ἀφίστημι,ἀφίσταιντο,,,,,,3rd,pl,[(ionic]
12133,ἀφίστημι,ἀφίσταιτο,,,,,,3rd,sg,[(ionic]
33678,ἀνά-ἐμπίπλημι,ἀνεμπίπλαιντο,,,,,,3rd,pl,[(ionic]
44794,ἀνακίρναμαι,ἀνακίρναιτο,,,,,,3rd,sg,[(ionic]
...,...,...,...,...,...,...,...,...,...,...
819622,ὑπεκτίθεμαι,ὑπεκθοῖντο,,,,,,3rd,pl,
837150,ὑποτίθημι,ὑποθέοιτο,,,,,,3rd,sg,[(ionic]
837185,ὑποτίθημι,ὑποθοίμεθα,,,,,,1st,pl,
837187,ὑποτίθημι,ὑποθοῖντο,,,,,,3rd,pl,


In [16]:
null_df = verb_df[verb_df["mode"] == " "]
null_df

,lemma,form,tense,mode,act/mid/p,gender,case,person,number,dialects
12131,ἀφίστημι,ἀφίσταιντ’,,,,,,3rd,pl,[(ionic]
12132,ἀφίστημι,ἀφίσταιντο,,,,,,3rd,pl,[(ionic]
12133,ἀφίστημι,ἀφίσταιτο,,,,,,3rd,sg,[(ionic]
33678,ἀνά-ἐμπίπλημι,ἀνεμπίπλαιντο,,,,,,3rd,pl,[(ionic]
44794,ἀνακίρναμαι,ἀνακίρναιτο,,,,,,3rd,sg,[(ionic]
...,...,...,...,...,...,...,...,...,...,...
819622,ὑπεκτίθεμαι,ὑπεκθοῖντο,,,,,,3rd,pl,
837150,ὑποτίθημι,ὑποθέοιτο,,,,,,3rd,sg,[(ionic]
837185,ὑποτίθημι,ὑποθοίμεθα,,,,,,1st,pl,
837187,ὑποτίθημι,ὑποθοῖντο,,,,,,3rd,pl,


In [18]:
null_df = verb_df[verb_df["act/mid/p"] == " "]
null_df

,lemma,form,tense,mode,act/mid/p,gender,case,person,number,dialects
107,ἅπτω,ἅβη,aor,ind,,,,3rd,sg,"[(doric, aeolic]"
108,ἅπτω,ἅβη,aor,ind,,,,3rd,sg,"[(homeric, ionic]"
109,ἅπτω,ἅφθη,aor,ind,,,,3rd,sg,"[(doric, aeolic]"
110,ἅπτω,ἅφθη,aor,ind,,,,3rd,sg,"[(homeric, ionic]"
336,ἅπτω,ἁφῆι,aor,subj,,,,3rd,sg,
...,...,...,...,...,...,...,...,...,...,...
859305,ζῳοποιέω1,ζῳοποιηθείη,aor,opt,,,,3rd,sg,
859306,ζῳοποιέω1,ζῳοποιηθείς,aor,part,,masc,nom/voc,,sg,
859307,ζῳοποιέω1,ζῳοποιηθήσονται,fut,ind,,,,3rd,pl,
859308,ζῳοποιέω1,ζῳοποιηθῶμεν,aor,subj,,,,1st,pl,[doric]


In [28]:
good_verb_df = verb_df[(verb_df["tense"]!= " ") & (verb_df["mode"]!=" ") & (verb_df["act/mid/p"]!= " ") & (verb_df["number"]!= " ")]
good_verb_df

,lemma,form,tense,mode,act/mid/p,gender,case,person,number,dialects
0,ἅλλομαι,ἅλεται,aor,subj,mid,,,3rd,sg,
1,ἅλλομαι,ἅληται,aor,subj,mid,,,3rd,sg,
2,ἅλλομαι,ἅλῃ,aor,subj,mid,,,2nd,sg,
4,ἅλλομαι,ἅλλεσθε,imperf,ind,mid,,,2nd,pl,"[(doric, aeolic]"
5,ἅλλομαι,ἅλλεσθε,pres,imperat,mid,,,2nd,pl,
...,...,...,...,...,...,...,...,...,...,...
859326,ζῳοτροφέω,ζῳοτροφεῖ,pres,ind,act,,,3rd,sg,"[doric, ionic]"
859328,ζῳοτροφέω,ζῳοτροφούντων,pres,part,act,masc/neut,gen,,pl,[doric]
859329,ζῳοτροφέω,ζῳοτροφούντων,pres,imperat,act,,,3rd,pl,[doric]
859330,ζῳοτροφέω,ζῳοτροφοῦσιν,pres,part,act,masc/neut,dat,,pl,[doric]


In [29]:
part_df = good_verb_df[good_verb_df["mode"] == "part"]
part_df

,lemma,form,tense,mode,act/mid/p,gender,case,person,number,dialects
28,ἅλλομαι,ἅλμενος,aor,part,mid,masc,nom,,sg,
43,ἅλλομαι,ἁλλόμενα,pres,part,mid,neut,nom/voc/acc,,pl,
44,ἅλλομαι,ἁλλόμεναι,pres,part,mid,fem,nom/voc,,pl,
45,ἅλλομαι,ἁλλόμενοι,pres,part,mid,masc,nom/voc,,pl,
46,ἅλλομαι,ἁλλόμενον,pres,part,mid,masc,acc,,sg,
...,...,...,...,...,...,...,...,...,...,...
859321,ζῳοποιέω1,ζῳοποιοῦσιν,pres,part,act,masc/neut,dat,,pl,[doric]
859323,ζῳοποιέω1,ζῳοποιουμένῃ,pres,part,mp,fem,dat,,sg,
859324,ζῳοποιέω1,ζῳοποιῶν,pres,part,act,masc,nom,,sg,[doric]
859328,ζῳοτροφέω,ζῳοτροφούντων,pres,part,act,masc/neut,gen,,pl,[doric]


In [30]:
unique_tense = good_verb_df['tense'].unique()
unique_mode = good_verb_df['mode'].unique()
unique_act = good_verb_df['act/mid/p'].unique()
unique_gender = good_verb_df['gender'].unique()
unique_case = good_verb_df['case'].unique()
unique_person = good_verb_df['person'].unique()
unique_number = good_verb_df['number'].unique()
unique_tense, unique_mode, unique_gender, unique_case, unique_person, unique_number

(array(['aor', 'imperf', 'pres', 'fut', 'perf', 'futperf', 'plup'],
       dtype=object),
 array(['subj', 'ind', 'imperat', 'opt', 'part'], dtype=object),
 array([' ', 'masc', 'neut', 'fem', 'masc/neut', 'masc/fem'], dtype=object),
 array([' ', 'nom', 'nom/voc/acc', 'nom/voc', 'acc', 'gen', 'dat', 'voc',
        'gen/dat', 'nom/acc'], dtype=object),
 array(['3rd', '2nd', '1st', ' '], dtype=object),
 array(['sg', 'pl', 'dual'], dtype=object))

In [32]:
unique_gender = noun_df['gender'].unique()
unique_case = noun_df['case'].unique()
unique_number = noun_df['number'].unique()
unique_gender, unique_case, unique_number

(array(['fem', 'masc', 'neut', 'masc/neut', 'masc/fem', 'masc/fem/neut',
        ' '], dtype=object),
 array(['nom/voc/acc', 'nom/voc', 'dat', 'acc', 'gen', 'nom', 'voc',
        'nom/acc', 'gen/dat'], dtype=object),
 array(['dual', 'sg', 'pl'], dtype=object))

In [59]:
noun_df

,lemma,form,gender,case,number,dialects
0,ἅβρα,ἅβρα,fem,nom/voc/acc,dual,
1,ἅβρα,ἅβρα,fem,nom/voc,sg,"[(attic, doric, aeolic]"
2,ἅβρα,ἅβραι,fem,nom/voc,pl,
3,ἅβρα,ἅβραι,fem,dat,sg,"[(attic, doric, aeolic]"
4,ἅβρα,ἅβραις,fem,dat,pl,
...,...,...,...,...,...,...
216247,ἵστωρ,ῐ̔́στορες,,nom,pl,
216248,ἵστωρ,ῐ̔́στωρ,,nom,sg,
216249,ἵστωρ,ῐ̔́στορε,,voc,dual,
216250,ἵστωρ,ῐ̔́στορες,,voc,pl,


In [62]:
good_noun_df = noun_df[(noun_df["gender"]!= " ") & (noun_df["case"]!=" ") & (noun_df["number"]!= " ")]

In [34]:
good_noun_df

,lemma,form,gender,case,number,dialects
0,ἅβρα,ἅβρα,fem,nom/voc/acc,dual,
1,ἅβρα,ἅβρα,fem,nom/voc,sg,"[(attic, doric, aeolic]"
2,ἅβρα,ἅβραι,fem,nom/voc,pl,
3,ἅβρα,ἅβραι,fem,dat,sg,"[(attic, doric, aeolic]"
4,ἅβρα,ἅβραις,fem,dat,pl,
...,...,...,...,...,...,...
216218,ἰσόψηφος,ἰσοψήφω,neut,voc,dual,
216219,ἰσόψηφος,ἰσόψηφοι,fem,voc,pl,
216220,ἰσόψηφος,ἰσόψηφε,fem,voc,sg,
216221,ἰσόψηφος,ἰσόψηφε,masc,voc,sg,


In [63]:
good_noun_df.insert(0, "POS", ["n" for x in good_noun_df.iterrows()])

In [64]:
good_noun_df = good_noun_df.drop("dialects", axis = 1)

In [48]:
good_noun_df

,POS,lemma,form,gender,case,number
0,n,ἅβρα,ἅβρα,fem,nom/voc/acc,dual
1,n,ἅβρα,ἅβρα,fem,nom/voc,sg
2,n,ἅβρα,ἅβραι,fem,nom/voc,pl
3,n,ἅβρα,ἅβραι,fem,dat,sg
4,n,ἅβρα,ἅβραις,fem,dat,pl
...,...,...,...,...,...,...
216218,n,ἰσόψηφος,ἰσοψήφω,neut,voc,dual
216219,n,ἰσόψηφος,ἰσόψηφοι,fem,voc,pl
216220,n,ἰσόψηφος,ἰσόψηφε,fem,voc,sg
216221,n,ἰσόψηφος,ἰσόψηφε,masc,voc,sg


In [65]:
new_rows = []

for id, row in good_noun_df.iterrows():
    if  "/" in row["case"]:
        cases = str(row["case"]).split("/")
        for case in cases:
            new_rows.append([row["POS"], row["lemma"], row["form"], row["gender"], case, row["number"]])
    
    if "/" in row["gender"]:
        genders = str(row["gender"]).split("/")
        for gender in genders:
            new_rows.append([row["POS"], row["lemma"], row["form"], gender, row["case"], row["number"]])

new_rows_df = new_rows_df = pd.DataFrame(new_rows, columns=["POS", "lemma", "form", "gender", "case", "number"])

good_noun_df = pd.concat([good_noun_df, new_rows_df], ignore_index=True)

In [66]:
x=good_noun_df[good_noun_df["form"]=="ἅβρα"]

In [67]:
x

,POS,lemma,form,gender,case,number
0,n,ἅβρα,ἅβρα,fem,nom/voc/acc,dual
1,n,ἅβρα,ἅβρα,fem,nom/voc,sg
213684,n,ἅβρα,ἅβρα,fem,nom,dual
213685,n,ἅβρα,ἅβρα,fem,voc,dual
213686,n,ἅβρα,ἅβρα,fem,acc,dual
213687,n,ἅβρα,ἅβρα,fem,nom,sg
213688,n,ἅβρα,ἅβρα,fem,voc,sg


In [68]:
# Remove rows where the "case" or "gender" columns contain "/"
good_noun_df = good_noun_df[~good_noun_df["case"].str.contains("/") & ~good_noun_df["gender"].str.contains("/")]

# Reset the index of the dataframe after filtering
good_noun_df = good_noun_df.reset_index(drop=True)

In [69]:
good_noun_df

,POS,lemma,form,gender,case,number
0,n,ἅβρα,ἅβραι,fem,dat,sg
1,n,ἅβρα,ἅβραις,fem,dat,pl
2,n,ἅβρα,ἅβραν,fem,acc,sg
3,n,ἅβρα,ἅβρας,fem,acc,pl
4,n,ἅβρα,ἅβρας,fem,gen,sg
...,...,...,...,...,...,...
329771,n,ζῳώδης,ζῳῶδες,masc,voc,sg
329772,n,ζῳώδης,ζῳῶδες,fem,voc,sg
329773,n,ζῳώδης,ζῳῶδες,neut,nom,sg
329774,n,ζῳώδης,ζῳῶδες,neut,voc,sg


In [38]:
good_verb_df

,lemma,form,tense,mode,act/mid/p,gender,case,person,number,dialects
0,ἅλλομαι,ἅλεται,aor,subj,mid,,,3rd,sg,
1,ἅλλομαι,ἅληται,aor,subj,mid,,,3rd,sg,
2,ἅλλομαι,ἅλῃ,aor,subj,mid,,,2nd,sg,
4,ἅλλομαι,ἅλλεσθε,imperf,ind,mid,,,2nd,pl,"[(doric, aeolic]"
5,ἅλλομαι,ἅλλεσθε,pres,imperat,mid,,,2nd,pl,
...,...,...,...,...,...,...,...,...,...,...
859326,ζῳοτροφέω,ζῳοτροφεῖ,pres,ind,act,,,3rd,sg,"[doric, ionic]"
859328,ζῳοτροφέω,ζῳοτροφούντων,pres,part,act,masc/neut,gen,,pl,[doric]
859329,ζῳοτροφέω,ζῳοτροφούντων,pres,imperat,act,,,3rd,pl,[doric]
859330,ζῳοτροφέω,ζῳοτροφοῦσιν,pres,part,act,masc/neut,dat,,pl,[doric]


In [42]:

good_verb_df.insert(0, "POS", ["v" if row["mode"] != "part" else "p" for id, row in good_verb_df.iterrows()])

In [70]:
good_verb_df = good_verb_df.drop("dialects", axis = 1)

In [71]:
good_verb_df

,POS,lemma,form,tense,mode,act/mid/p,gender,case,person,number
0,v,ἅλλομαι,ἅλεται,aor,subj,mid,,,3rd,sg
1,v,ἅλλομαι,ἅληται,aor,subj,mid,,,3rd,sg
2,v,ἅλλομαι,ἅλῃ,aor,subj,mid,,,2nd,sg
4,v,ἅλλομαι,ἅλλεσθε,imperf,ind,mid,,,2nd,pl
5,v,ἅλλομαι,ἅλλεσθε,pres,imperat,mid,,,2nd,pl
...,...,...,...,...,...,...,...,...,...,...
859326,v,ζῳοτροφέω,ζῳοτροφεῖ,pres,ind,act,,,3rd,sg
859328,p,ζῳοτροφέω,ζῳοτροφούντων,pres,part,act,masc/neut,gen,,pl
859329,v,ζῳοτροφέω,ζῳοτροφούντων,pres,imperat,act,,,3rd,pl
859330,p,ζῳοτροφέω,ζῳοτροφοῦσιν,pres,part,act,masc/neut,dat,,pl


In [72]:
new_rows = []

for id, row in good_verb_df.iterrows():
    if  "/" in row["case"]:
        cases = str(row["case"]).split("/")
        for case in cases:
            new_rows.append([row["POS"], row["lemma"], row["form"], row["tense"], row["mode"], row["act/mid/p"], row["gender"], case, row["person"], row["number"]])
    
    if "/" in row["gender"]:
        genders = str(row["gender"]).split("/")
        for gender in genders:
            new_rows.append([row["POS"], row["lemma"], row["form"],row["tense"], row["mode"], row["act/mid/p"], gender, row["case"], row["person"], row["number"]])

new_rows_df = new_rows_df = pd.DataFrame(new_rows, columns=["POS", "lemma", "form", "tense", "mode", "act/mid/p", "gender", "case", "person", "number"])

good_verb_df = pd.concat([good_verb_df, new_rows_df], ignore_index=True)

In [74]:
# Remove rows where the "case" or "gender" columns contain "/"
good_verb_df = good_verb_df[~good_verb_df["case"].str.contains("/") & ~good_verb_df["gender"].str.contains("/")]

# Reset the index of the dataframe after filtering
good_verb_df = good_verb_df.reset_index(drop=True)

In [75]:
good_verb_df

,POS,lemma,form,tense,mode,act/mid/p,gender,case,person,number
0,v,ἅλλομαι,ἅλεται,aor,subj,mid,,,3rd,sg
1,v,ἅλλομαι,ἅληται,aor,subj,mid,,,3rd,sg
2,v,ἅλλομαι,ἅλῃ,aor,subj,mid,,,2nd,sg
3,v,ἅλλομαι,ἅλλεσθε,imperf,ind,mid,,,2nd,pl
4,v,ἅλλομαι,ἅλλεσθε,pres,imperat,mid,,,2nd,pl
...,...,...,...,...,...,...,...,...,...,...
967083,p,ζῳοποιέω1,ζῳοποιοῦσιν,pres,part,act,neut,dat,,pl
967084,p,ζῳοτροφέω,ζῳοτροφούντων,pres,part,act,masc,gen,,pl
967085,p,ζῳοτροφέω,ζῳοτροφούντων,pres,part,act,neut,gen,,pl
967086,p,ζῳοτροφέω,ζῳοτροφοῦσιν,pres,part,act,masc,dat,,pl


In [83]:
morph_df = pd.concat([good_verb_df, good_noun_df])
morph_df

,POS,lemma,form,tense,mode,act/mid/p,gender,case,person,number
0,v,ἅλλομαι,ἅλεται,aor,subj,mid,,,3rd,sg
1,v,ἅλλομαι,ἅληται,aor,subj,mid,,,3rd,sg
2,v,ἅλλομαι,ἅλῃ,aor,subj,mid,,,2nd,sg
3,v,ἅλλομαι,ἅλλεσθε,imperf,ind,mid,,,2nd,pl
4,v,ἅλλομαι,ἅλλεσθε,pres,imperat,mid,,,2nd,pl
...,...,...,...,...,...,...,...,...,...,...
329771,n,ζῳώδης,ζῳῶδες,NaN,NaN,NaN,masc,voc,NaN,sg
329772,n,ζῳώδης,ζῳῶδες,NaN,NaN,NaN,fem,voc,NaN,sg
329773,n,ζῳώδης,ζῳῶδες,NaN,NaN,NaN,neut,nom,NaN,sg
329774,n,ζῳώδης,ζῳῶδες,NaN,NaN,NaN,neut,voc,NaN,sg


In [78]:
i = morph_df[(morph_df["case"].str.contains("/")) & (morph_df["gender"].str.contains("/"))]

,POS,lemma,form,tense,mode,act/mid/p,gender,case,person,number


In [84]:
morph_df

,POS,lemma,form,tense,mode,act/mid/p,gender,case,person,number
0,v,ἅλλομαι,ἅλεται,aor,subj,mid,,,3rd,sg
1,v,ἅλλομαι,ἅληται,aor,subj,mid,,,3rd,sg
2,v,ἅλλομαι,ἅλῃ,aor,subj,mid,,,2nd,sg
3,v,ἅλλομαι,ἅλλεσθε,imperf,ind,mid,,,2nd,pl
4,v,ἅλλομαι,ἅλλεσθε,pres,imperat,mid,,,2nd,pl
...,...,...,...,...,...,...,...,...,...,...
329771,n,ζῳώδης,ζῳῶδες,NaN,NaN,NaN,masc,voc,NaN,sg
329772,n,ζῳώδης,ζῳῶδες,NaN,NaN,NaN,fem,voc,NaN,sg
329773,n,ζῳώδης,ζῳῶδες,NaN,NaN,NaN,neut,nom,NaN,sg
329774,n,ζῳώδης,ζῳῶδες,NaN,NaN,NaN,neut,voc,NaN,sg


In [80]:
from sklearn.preprocessing import LabelEncoder

In [85]:
features = ["POS", "tense", "mode", "act/mid/p", "gender", "case", "person", "number"]

label_encoders = {feature: LabelEncoder() for feature in features}

for feature in features:
    morph_df[feature] = label_encoders[feature].fit_transform(morph_df[feature].fillna('None'))

morph_df


,POS,lemma,form,tense,mode,act/mid/p,gender,case,person,number
0,2,ἅλλομαι,ἅλεται,1,5,2,0,0,3,2
1,2,ἅλλομαι,ἅληται,1,5,2,0,0,3,2
2,2,ἅλλομαι,ἅλῃ,1,5,2,0,0,2,2
3,2,ἅλλομαι,ἅλλεσθε,4,2,2,0,0,2,1
4,2,ἅλλομαι,ἅλλεσθε,7,1,2,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...
329771,0,ζῳώδης,ζῳῶδες,0,0,0,2,5,4,2
329772,0,ζῳώδης,ζῳῶδες,0,0,0,1,5,4,2
329773,0,ζῳώδης,ζῳῶδες,0,0,0,3,4,4,2
329774,0,ζῳώδης,ζῳῶδες,0,0,0,3,5,4,2


In [94]:
temp = morph_df[(morph_df["lemma"].str.contains("=")) | (morph_df["form"].str.contains("="))]
temp

,POS,lemma,form,tense,mode,act/mid/p,gender,case,person,number
203002,2,ἐνοικο=δομέω,ἐνοικοδομεῖται,7,2,3,0,0,3,2
203003,1,ἐνοικο=δομέω,ἐνοικοδομήσαντα,1,4,1,2,1,0,2
203004,2,ἐνοικο=δομέω,ἐνοικοδομήσει,1,5,1,0,0,3,2
203005,2,ἐνοικο=δομέω,ἐνοικοδομήσει,2,2,2,0,0,2,2
203006,2,ἐνοικο=δομέω,ἐνοικοδομήσει,2,2,1,0,0,3,2
203007,2,ἐνοικο=δομέω,ἐνοικοδομήσηται,1,5,2,0,0,3,2
203008,2,ἐνοικο=δομέω,ἐνοικοδομήσῃ,1,5,2,0,0,2,2
203009,2,ἐνοικο=δομέω,ἐνοικοδομήσῃ,1,5,1,0,0,3,2
203010,2,ἐνοικο=δομέω,ἐνοικοδομήσῃ,2,2,2,0,0,2,2
203011,2,ἐνοικο=δομέω,ἐνοικοδομήσομεν,1,5,1,0,0,1,1


In [86]:
import string

In [87]:
all_characters = set()

def collect_unique_chars(word):
    for char in word:
        all_characters.add(char)

morph_df["lemma"].apply(collect_unique_chars)
morph_df["form"].apply(collect_unique_chars)

char_vocab = (sorted(list(all_characters)))
char_vocab

['%',
 '(',
 ')',
 ',',
 '-',
 '/',
 '1',
 '2',
 '3',
 '=',
 '́',
 'Λ',
 'Μ',
 'Σ',
 'Χ',
 'ά',
 'έ',
 'ή',
 'ί',
 'α',
 'β',
 'γ',
 'δ',
 'ε',
 'ζ',
 'η',
 'θ',
 'ι',
 'κ',
 'λ',
 'μ',
 'ν',
 'ξ',
 'ο',
 'π',
 'ρ',
 'ς',
 'σ',
 'τ',
 'υ',
 'φ',
 'χ',
 'ψ',
 'ω',
 'ϊ',
 'ϋ',
 'ό',
 'ύ',
 'ώ',
 'ἀ',
 'ἁ',
 'ἄ',
 'ἅ',
 'ἆ',
 'ἇ',
 'Ἀ',
 'ἐ',
 'ἑ',
 'ἔ',
 'ἕ',
 'Ἐ',
 'Ἑ',
 'ἠ',
 'ἡ',
 'ἤ',
 'ἥ',
 'ἦ',
 'ἧ',
 'ἰ',
 'ἱ',
 'ἴ',
 'ἵ',
 'ἶ',
 'ἷ',
 'ὀ',
 'ὁ',
 'ὄ',
 'ὅ',
 'ὐ',
 'ὑ',
 'ὔ',
 'ὕ',
 'ὖ',
 'ὗ',
 'ὠ',
 'ὡ',
 'ὤ',
 'ὥ',
 'ὦ',
 'ὧ',
 'ὶ',
 'ᾀ',
 'ᾄ',
 'ᾅ',
 'ᾆ',
 'ᾇ',
 'ᾐ',
 'ᾑ',
 'ᾔ',
 'ᾕ',
 'ᾖ',
 'ᾗ',
 'ᾠ',
 'ᾡ',
 'ᾤ',
 'ᾦ',
 'ᾧ',
 'ᾰ',
 'ᾱ',
 'ᾳ',
 'ᾴ',
 'ᾶ',
 'ᾷ',
 'ῃ',
 'ῄ',
 'ῆ',
 'ῇ',
 'ῐ',
 'ΐ',
 'ῖ',
 'ῠ',
 'ῥ',
 'ῦ',
 'ῳ',
 'ῴ',
 'ῶ',
 'ῷ',
 '’',
 '†']

In [82]:
label_encoders

{'POS': LabelEncoder(),
 'lemma': LabelEncoder(),
 'form': LabelEncoder(),
 'tense': LabelEncoder(),
 'mode': LabelEncoder(),
 'act/mid/p': LabelEncoder(),
 'gender': LabelEncoder(),
 'case': LabelEncoder(),
 'person': LabelEncoder(),
 'number': LabelEncoder()}

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, TimeDistributed, Concatenate

# Parameters
lemma_vocab_size = 20000  # adjust based on your vocabulary size for lemmas
tag_vocab_size = 500     # adjust based on your total number of unique morphological tags
lemma_embedding_dim = 256
tag_embedding_dim = 50   # smaller dimension for categorical data
lstm_units = 256

# Define inputs
lemma_inputs = Input(shape=(None,))  # Input for lemmas
tag_inputs = Input(shape=(None,))    # Input for morphological tags

# Embeddings
lemma_embedding = Embedding(lemma_vocab_size, lemma_embedding_dim)(lemma_inputs)
tag_embedding = Embedding(tag_vocab_size, tag_embedding_dim)(tag_inputs)

# Combine the embeddings
combined_embeddings = Concatenate()([lemma_embedding, tag_embedding])

# Encoder LSTM
encoder_lstm = LSTM(lstm_units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(combined_embeddings)
encoder_states = [state_h, state_c]

# Decoder setup
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(lemma_vocab_size, lemma_embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = TimeDistributed(Dense(lemma_vocab_size, activation='softmax'))
decoder_outputs = decoder_dense(decoder_outputs)

# Define and create the model
model = Model([lemma_inputs, tag_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
